# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 14 2023 9:55AM,259247,10,PRF-55378,Bio-PRF,Released
1,Mar 14 2023 9:55AM,259247,10,PRF-55381,Bio-PRF,Released
2,Mar 14 2023 9:55AM,259247,10,PRF-55384,Bio-PRF,Released
3,Mar 14 2023 9:55AM,259247,10,PRF-55387,Bio-PRF,Released
4,Mar 14 2023 9:55AM,259247,10,PRF-55390,Bio-PRF,Released
5,Mar 14 2023 9:55AM,259247,10,PRF-55393,Bio-PRF,Released
6,Mar 14 2023 9:55AM,259247,10,PRF-55425,Bio-PRF,Released
7,Mar 14 2023 9:55AM,259247,10,PRF-55428,Bio-PRF,Released
8,Mar 14 2023 9:55AM,259247,10,PRF-55396,Bio-PRF,Released
9,Mar 14 2023 9:55AM,259247,10,PRF-55399,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
10,259246,Executing,1
11,259247,Released,14
12,259248,Released,1
13,259249,Released,1
14,259250,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
259246,1.0,NaN
259247,NaN,14.0
259248,NaN,1.0
259249,NaN,1.0
259250,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
259154,1.0,0.0
259195,0.0,64.0
259200,0.0,2.0
259215,0.0,10.0
259221,11.0,11.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
259154,1,0
259195,0,64
259200,0,2
259215,0,10
259221,11,11


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,259154,1,0
1,259195,0,64
2,259200,0,2
3,259215,0,10
4,259221,11,11


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,259154,1,
1,259195,,64
2,259200,,2
3,259215,,10
4,259221,11,11


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 14 2023 9:55AM,259247,10,Bio-PRF
14,Mar 14 2023 9:48AM,259215,15,"Alliance Pharma, Inc."
24,Mar 14 2023 9:46AM,259195,15,"Alliance Pharma, Inc."
88,Mar 14 2023 9:43AM,259250,18,"Emersa Waterbox, LLC"
89,Mar 14 2023 9:36AM,259249,10,Hush Hush
90,Mar 14 2023 9:29AM,259248,16,Sartorius Bioprocess Solutions
91,Mar 14 2023 9:12AM,259246,12,HealthCaps LLC
92,Mar 14 2023 9:07AM,259245,10,"Digital Brands, LLC"
102,Mar 14 2023 8:52AM,259243,10,Hush Hush
118,Mar 14 2023 8:33AM,259225,20,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 14 2023 9:55AM,259247,10,Bio-PRF,,14
1,Mar 14 2023 9:48AM,259215,15,"Alliance Pharma, Inc.",,10
2,Mar 14 2023 9:46AM,259195,15,"Alliance Pharma, Inc.",,64
3,Mar 14 2023 9:43AM,259250,18,"Emersa Waterbox, LLC",,1
4,Mar 14 2023 9:36AM,259249,10,Hush Hush,,1
5,Mar 14 2023 9:29AM,259248,16,Sartorius Bioprocess Solutions,,1
6,Mar 14 2023 9:12AM,259246,12,HealthCaps LLC,1,
7,Mar 14 2023 9:07AM,259245,10,"Digital Brands, LLC",,10
8,Mar 14 2023 8:52AM,259243,10,Hush Hush,16,
9,Mar 14 2023 8:33AM,259225,20,"Exact-Rx, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 14 2023 9:55AM,259247,10,Bio-PRF,14,
1,Mar 14 2023 9:48AM,259215,15,"Alliance Pharma, Inc.",10,
2,Mar 14 2023 9:46AM,259195,15,"Alliance Pharma, Inc.",64,
3,Mar 14 2023 9:43AM,259250,18,"Emersa Waterbox, LLC",1,
4,Mar 14 2023 9:36AM,259249,10,Hush Hush,1,
5,Mar 14 2023 9:29AM,259248,16,Sartorius Bioprocess Solutions,1,
6,Mar 14 2023 9:12AM,259246,12,HealthCaps LLC,,1
7,Mar 14 2023 9:07AM,259245,10,"Digital Brands, LLC",10,
8,Mar 14 2023 8:52AM,259243,10,Hush Hush,,16
9,Mar 14 2023 8:33AM,259225,20,"Exact-Rx, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 14 2023 9:55AM,259247,10,Bio-PRF,14,
1,Mar 14 2023 9:48AM,259215,15,"Alliance Pharma, Inc.",10,
2,Mar 14 2023 9:46AM,259195,15,"Alliance Pharma, Inc.",64,
3,Mar 14 2023 9:43AM,259250,18,"Emersa Waterbox, LLC",1,
4,Mar 14 2023 9:36AM,259249,10,Hush Hush,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 14 2023 9:55AM,259247,10,Bio-PRF,14.0,NaN
1,Mar 14 2023 9:48AM,259215,15,"Alliance Pharma, Inc.",10.0,NaN
2,Mar 14 2023 9:46AM,259195,15,"Alliance Pharma, Inc.",64.0,NaN
3,Mar 14 2023 9:43AM,259250,18,"Emersa Waterbox, LLC",1.0,NaN
4,Mar 14 2023 9:36AM,259249,10,Hush Hush,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 14 2023 9:55AM,259247,10,Bio-PRF,14.0,0.0
1,Mar 14 2023 9:48AM,259215,15,"Alliance Pharma, Inc.",10.0,0.0
2,Mar 14 2023 9:46AM,259195,15,"Alliance Pharma, Inc.",64.0,0.0
3,Mar 14 2023 9:43AM,259250,18,"Emersa Waterbox, LLC",1.0,0.0
4,Mar 14 2023 9:36AM,259249,10,Hush Hush,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1296184,27.0,16.0
12,259246,0.0,1.0
15,518410,74.0,0.0
16,259248,1.0,0.0
18,518404,1.0,1.0
19,518443,11.0,12.0
20,259225,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1296184,27.0,16.0
1,12,259246,0.0,1.0
2,15,518410,74.0,0.0
3,16,259248,1.0,0.0
4,18,518404,1.0,1.0
5,19,518443,11.0,12.0
6,20,259225,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,27.0,16.0
1,12,0.0,1.0
2,15,74.0,0.0
3,16,1.0,0.0
4,18,1.0,1.0
5,19,11.0,12.0
6,20,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,27.0
1,12,Released,0.0
2,15,Released,74.0
3,16,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20
Status,,,,,,,
Executing,16.0,1.0,0.0,0.0,1.0,12.0,0.0
Released,27.0,0.0,74.0,1.0,1.0,11.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20
0,Executing,16.0,1.0,0.0,0.0,1.0,12.0,0.0
1,Released,27.0,0.0,74.0,1.0,1.0,11.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20
0,Executing,16.0,1.0,0.0,0.0,1.0,12.0,0.0
1,Released,27.0,0.0,74.0,1.0,1.0,11.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()